## About webdocs
- reading webdocs.dat ... [5267656 item(s), 1692082 transaction(s)]
- sorting and reducing transactions ... [959755/1692082 transaction(s)]

- Distinct element: 5267656
- Number of incomming transactions: 1692082
- Top-10 frequent element and count: 

| Element | Count |
| :----- | :----- |
|	122	|1429525|
|	8	|1309694|
|	49	|1178279|
|	124	|887358|
|	516	|858146|
|	171	|793840|
|	51	|787398|
|	121	|726516|
|	878	|677779|
|	150	|677637|

## 配置一
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
沒有keep

    Top-512,Sketch:128*256
    Execution time:1265.818 seconds.
    Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
    Precision:   0.9434
    Find:483, TP:483, FP:29
    Top_ARE: 0.009426
    Top_AAE: 62.007812
    all_ARE: 10.356128
    all_AAE: 25.525622
    Estimate time:  40.263 seconds.


加入Keep  

    Top-512,Sketch:128*256
    Execution time:1030.936 seconds.
    Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
    Precision:   0.9492
    Find:486, TP:486, FP:26
    Top_ARE: 0.008511
    Top_AAE: 57.750000
    all_ARE: 10.443191
    all_AAE: 25.509736
    Estimate time:  35.904 seconds.
        
## 配置二     
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
    hash2=mmh3.hash(element.ID, signed=False)  
沒有keep

    Top-512,Sketch:128*256
    Execution time:1024.824 seconds.
    Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
    Precision:   0.9375
    Find:480, TP:480, FP:32
    Top_ARE: 0.009045
    Top_AAE: 62.347656
    all_ARE: 10.357845
    all_AAE: 25.455249
    Estimate time:  37.468 seconds.

加入Keep

    Top-512,Sketch:128*256
    Execution time:1024.330 seconds.
    Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
    Precision:   0.9395
    Find:481, TP:481, FP:31
    Top_ARE: 0.007369
    Top_AAE: 49.697266
    all_ARE: 10.375286
    all_AAE: 25.446483
    Estimate time:  35.855 seconds.


## ok的版本, 沒有keep

In [108]:
import numpy as np
import spookyhash
import mmh3
import os
import pandas as pd
import time
import operator
import hyperloglog
import sys
import re


# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=get_emax()
    width,depth=get_width_depth()
    col,row=position(element)
        # col / row index of element 
    avg=0
    #print("{} send to Sk[{}][{}]".format(element,row,col))
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    Sk_head[row].distinct.add(element.ID)
    Sk[row][col]+=1

    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk,row)

'''
    print("e_max:{}".format(e_max))
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print('')
'''


# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # local max need only 1 row
    #print("In Update_local_max:")
    width,depth=get_width_depth()
    if head_item.maxID=='':
        head_item.maxID=element.ID
    else:
        local_max_col,local_max_row=position(Tail(head_item.maxID,0))
        if element_list[local_max_col]<element_list[column]:
            head_item.maxID=element.ID


# ==========================update e_max==========================
def Update_emax(head,sketch,sk_row):
    e_max=get_emax()
    local_max_col,local_max_row=position(Tail(head[sk_row].maxID,0))
    if sketch[local_max_row][local_max_col]>e_max.count:
        e_max.ID=head[sk_row].maxID
        e_max.count=sketch[local_max_row][local_max_col]
    
    '''
    # pass whole array
    #print("In Update_emax:")
    e_max=get_emax()
    for i in range(len(head)):
        if head[i].maxID=='':
            continue
        else:
            local_max_col,local_max_row=position(Tail(head[i].maxID,0))
            if sketch[local_max_row][local_max_col]>e_max.count:
                e_max.ID=head[i].maxID
                e_max.count=sketch[local_max_row][local_max_col]   
    
    '''


# ========================== BringBack=========================
def BringBack(e_min,head,sketch):
    # bring e_max back to Top
    # e_min=e_max, e_max=Null, delete e_max.count in Sketch, send e_min into Sketch
    e_max=get_emax()
    temp=Tail(e_min.ID,e_min.count)
    e_min.ID=e_max.ID
    e_min.count=e_max.count
    DeleteSk(e_max,head,sketch)
    UpdateSk(temp,head,sketch)

# ==========================DeleteSk=========================
def DeleteSk(element,head,sketch):
    # e_max in sketch: sketch[r][c]=0, total count-=sketch[row][col]
    width,depth=get_width_depth()
    col,row=position(element)
    head[row].count-=e_max.count
        # total_count-=element.count
    sketch[row][col]=0
    head[row].maxID=''
    element.ID=""
    element.count=0
# ==========================Tools=========================    
def get_emax():
    return e_max
def get_width_depth():
    return width,depth

def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    width,depth=get_width_depth()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input: byte
        # output:unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
    #hash2=mmh3.hash(element.ID, signed=False)
        # input: str
        # output: unsigned- 32 bit int
    col=hash2 % width
    row=hash1 % depth
    return col,row 

def Query(e,Top,local_maxID,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(Tail(e,0))
        if e in local_maxID:
            count=Sketch[row][col]
        else:
            ratio=width/row_cardinality[row]
            count=int(Sketch[row][col]*ratio)
    if count<=1:
        count=1
    return count
# ==========================main=========================    

#path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
# path=r'C:\Users\Pero\python\Python\webdocs'
datapath='..\dataset\webdocs'
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(datapath)))
    # filelist
depth=128
width=256
size=512
Sk_head=[Head(0) for j in range(depth)]
Sketch=np.zeros((depth,width),dtype='int32')
e_max=Tail('',0)
Top=[]

item_count=1000
income=0

start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(datapath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                #item_count-=1
                #income+=1
                #print("read {}-th transaction:{}".format(income,eli))            
                item=Tail(e,1)
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                        index=len(Top)-1
                    else:
                        UpdateSk(item,Sk_head,Sketch)
                else:
                    Top[index].count+=1                
                    if index==0 or Top[index].count< Top[index-1].count:
                        pass
                    else:
                        Top.sort(key=operator.attrgetter('count'),reverse=True)                
                if e_max.count>Top[-1].count:
                    BringBack(Top[-1],Sk_head,Sketch)
                    Top.sort(key=operator.attrgetter('count'),reverse=True)
                    #print('Top after BringBack: \n\t{}'.format(Top)) 

end=time.time()
print("Top-{},Sketch:{}*{}".format(size,depth,width))
print("Execution time:{:8.3f} seconds.".format(end-start))
print("Total memory {} bytes=".format(sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth),end='')
print("Top:{} bytes, Sketch:{} bytes, Sketch_head:{} bytes.".format(sys.getsizeof(Top),Sketch.nbytes,sys.getsizeof(Sk_head[0])*depth))

'''
print("TOP[20]:\n{}".format(Top[:20]))
print("e_max:{}".format(e_max))
for i in range(len(Sketch)):
    print("Sk[{}]:{},{}".format(i,Sk_head[i],Sketch[i]))
print('')

'''
#====================Top to csv=============================
templi=[[i.ID,i.count] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])
df=df.sort_values(by='Count',ascending=False)
    # 依count由大到小排序
name="My_webdocs_Top-"+str(size)+'_'+str(depth)+'_'+str(width)+'.csv'
df.to_csv(os.path.join(r'..\result',name),index=False)
    #儲存Top的結果

#====================result compare=============================
gr_path='..\dataset\webdocs'
gr_file_name='webdocs_00_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_path,gr_file_name))
My_result=pd.read_csv(os.path.join(r'..\result',name))

# precision
gt_set=set(grtruth['Element'][:size])
    # Top-k ground truth
my_set=set(My_result['ID'])
tp_set=gt_set & my_set
    # true-positive set
    # &: set 交集運算
precision=len(tp_set)/len(my_set)
print("Precision: {:8.4f}".format(precision))

# ====================ARE/AAE for top and all====================
gt_dict=dict(grtruth.values.tolist())
top_dict=dict(My_result.values.tolist())
distinct=len(gt_dict)
    # cardinality of all incoming elements
row_cardinality=[len(i.distinct) for i in Sk_head]
    # number of distinct elements in each row in Sk_head
local_maxID=[i.maxID for i in Sk_head]
    # list of local max elements in each row of Sketch
all_are=0
all_aae=0
top_are=0
top_aae=0
tp=0
fp=0

startx=time.time()
for item in gt_dict:
    count=Query(str(item),Top,local_maxID,Sketch,row_cardinality)
    all_are+=abs(count-gt_dict[item])/gt_dict[item]
    all_aae+=abs(count-gt_dict[item])
    if item in top_dict:
        # compute ARE/AAE of all elements in Top but not only hit elements
        top_are+=abs(count-gt_dict[item])/gt_dict[item]
        top_aae+=abs(count-gt_dict[item])
        if item in tp_set:
            tp+=1
        else:
            fp+=1
endx=time.time()

top_are=top_are/size
top_aae=top_aae/size
all_are=all_are/distinct
all_aae=all_aae/distinct
print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))        

# ====================Cover result into a dataframe====================
sketch_size=str(depth)+'*'+str(width)
temp=sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth
memory_usage=str(temp)+' bytes ='+'Top:'+str(sys.getsizeof(Top))+'+ Sketch:'+str(Sketch.nbytes)+'+ Sk_head:'+str(sys.getsizeof(Sk_head[0])*depth)

result_df=pd.DataFrame(columns=['Top-k',
                                'Sketch',
                                'Total memory',
                                'Exe_time',
                                'Find',
                                'Precision',
                                'ARE-Top',
                                'AAE-Top',
                                'ARE-all',
                                'AAE-all'])
output_dict={
    'Top-k':size,
    'Sketch':sketch_size,
    'Total memory':memory_usage,
    'Exe_time':float('{:.3f}'.format(end-start)),
    'Find':"Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp),
    'Precision':float("{:8.4f}".format(precision)),
    'ARE-Top':float('{:8.6f}'.format(top_are)),
    'AAE-Top':float('{:8.6f}'.format(top_aae)),
    'ARE-all':float('{:8.6f}'.format(all_ARE)),
    'AAE-all':float('{:8.6f}'.format(all_AAE))}

result_df=result_df.append(output_dict,ignore_index=True)
file="My_webdocs_result"+'_'+str(size)+'_'+str(depth)+'_'+str(width)+'_.csv'
    # 程式執行結果供複製用
result_df.to_csv(os.path.join(os.path.join(r'..\result',file)),index=False)
result_df

EOF
Top-512,Sketch:128*256
Execution time:1265.818 seconds.
Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
Precision:   0.9434
Find:483, TP:483, FP:29
Top_ARE: 0.009426
Top_AAE: 62.007812
all_ARE: 10.356128
all_AAE: 25.525622
Estimate time:  40.263 seconds.


,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,512,128*256,142512 bytes =Top:4272+ Sketch:131072+ Sk_head...,1265.818,"Find:483, TP:483, FP:29",0.9434,0.009426,62.007812,10.70181,26.593554


In [ ]:
y=xdf.groupby(by='Count')
z=y.size()
    # only 139 different count for this dataset
print(z.index.tolist())
    # e出現的頻率
print(z.values.tolist())
    # 不同頻率出現的次數

## with Keep

In [107]:
import numpy as np
import spookyhash
import mmh3
import os
import pandas as pd
import time
import operator
import hyperloglog
import sys
import re


# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
        self.keep=0
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}, keep: {}]'.format(self.count,len(self.distinct),self.maxID,self.keep)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}, keep: {}]'.format(self.count,len(self.distinct),self.maxID,self.keep)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=get_emax()
    width,depth=get_width_depth()
    col,row=position(element)
        # col / row index of element 
    avg=0
    #print("{} send to Sk[{}][{}]".format(element,row,col))
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    Sk_head[row].distinct.add(element.ID)
    Sk[row][col]+=1

    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk,row)

'''
    print("e_max:{}".format(e_max))
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print('')
'''


# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # local max need only 1 row
    #print("In Update_local_max:")
    width,depth=get_width_depth()
    if head_item.maxID=='':
        head_item.maxID=element.ID
        head_item.keep=1
    elif head_item.maxID==element.ID:
        head_item.keep+=1
    else:
        # head_item.maxID!=element.ID:
        local_max_col,local_max_row=position(Tail(head_item.maxID,0))
        if local_max_col==column:
            # local_max collision with e
            head_item.keep-=1
            if head_item.keep<=0:
                head_item.maxID=element.ID
        elif element_list[local_max_col]<element_list[column]:
            # no collision
            head_item.maxID=element.ID
            head_item.keep=1


# ==========================update e_max==========================
def Update_emax(head,sketch,sk_row):
    e_max=get_emax()
    local_max_col,local_max_row=position(Tail(head[sk_row].maxID,0))
    if sketch[local_max_row][local_max_col]>e_max.count:
        e_max.ID=head[sk_row].maxID
        e_max.count=sketch[local_max_row][local_max_col]
    
    '''
    # pass whole array
    #print("In Update_emax:")
    e_max=get_emax()
    for i in range(len(head)):
        if head[i].maxID=='':
            continue
        else:
            local_max_col,local_max_row=position(Tail(head[i].maxID,0))
            if sketch[local_max_row][local_max_col]>e_max.count:
                e_max.ID=head[i].maxID
                e_max.count=sketch[local_max_row][local_max_col]   
    
    '''


# ========================== BringBack=========================
def BringBack(e_min,head,sketch):
    # bring e_max back to Top
    # e_min=e_max, e_max=Null, delete e_max.count in Sketch, send e_min into Sketch
    e_max=get_emax()
    temp=Tail(e_min.ID,e_min.count)
    e_min.ID=e_max.ID
    e_min.count=e_max.count
    DeleteSk(e_max,head,sketch)
    UpdateSk(temp,head,sketch)

# ==========================DeleteSk=========================
def DeleteSk(element,head,sketch):
    # e_max in sketch: sketch[r][c]=0, total count-=sketch[row][col]
    width,depth=get_width_depth()
    col,row=position(element)
    head[row].count-=e_max.count
        # total_count-=element.count
    sketch[row][col]=0
    head[row].maxID=''
    element.ID=""
    element.count=0
# ==========================Tools=========================    
def get_emax():
    return e_max
def get_width_depth():
    return width,depth

def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    width,depth=get_width_depth()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input: byte
        # output:unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
    #hash2=mmh3.hash(element.ID, signed=False)
        # input: str
        # output: unsigned- 32 bit int
    col=hash2 % width
    row=hash1 % depth
    return col,row 

def Query(e,Top,local_maxID,Sketch,row_cardinality):
    # e is ID, type(e)=str
    # row_cardinality is list of number of distinct elements in each row in Sk_head
    # local_maxID is list of local max element in each row of Sketch
    count=0
    index=find(Tail(e,0),Top)
    if index>=0:
        # e in Top
        count=Top[index].count
    else:
        # e in Sketch
        col,row=position(Tail(e,0))
        if e in local_maxID:
            count=Sketch[row][col]
        else:
            ratio=width/row_cardinality[row]
            count=int(Sketch[row][col]*ratio)
    if count<=1:
        count=1
    return count
# ==========================main=========================    

#path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
# path=r'C:\Users\Pero\python\Python\webdocs'
datapath='..\dataset\webdocs'
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(datapath)))
    # filelist
depth=128
width=256
size=512
Sk_head=[Head(0) for j in range(depth)]
Sketch=np.zeros((depth,width),dtype='int32')
e_max=Tail('',0)
Top=[]

item_count=1000
income=0

start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(datapath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                #item_count-=1
                #income+=1
                #print("read {}-th transaction:{}".format(income,eli))            
                item=Tail(e,1)
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                        index=len(Top)-1
                    else:
                        UpdateSk(item,Sk_head,Sketch)
                else:
                    Top[index].count+=1                
                    if index==0 or Top[index].count< Top[index-1].count:
                        pass
                    else:
                        Top.sort(key=operator.attrgetter('count'),reverse=True)                
                if e_max.count>Top[-1].count:
                    BringBack(Top[-1],Sk_head,Sketch)
                    Top.sort(key=operator.attrgetter('count'),reverse=True)
                    #print('Top after BringBack: \n\t{}'.format(Top)) 

end=time.time()
print("Top-{},Sketch:{}*{}".format(size,depth,width))
print("Execution time:{:8.3f} seconds.".format(end-start))
print("Total memory {} bytes=".format(sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth),end='')
print("Top:{} bytes, Sketch:{} bytes, Sketch_head:{} bytes.".format(sys.getsizeof(Top),Sketch.nbytes,sys.getsizeof(Sk_head[0])*depth))

'''
print("TOP[20]:\n{}".format(Top[:20]))
print("e_max:{}".format(e_max))
for i in range(len(Sketch)):
    print("Sk[{}]:{},{}".format(i,Sk_head[i],Sketch[i]))
print('')

'''
#====================Top to csv=============================
templi=[[i.ID,i.count] for i in Top]
df=pd.DataFrame(templi,columns=['ID', 'Count'])
df=df.sort_values(by='Count',ascending=False)
    # 依count由大到小排序
name="My_webdocs_Top-"+str(size)+'_'+str(depth)+'_'+str(width)+'.csv'
df.to_csv(os.path.join(r'..\result',name),index=False)
    #儲存Top的結果

#====================result compare=============================
gr_path='..\dataset\webdocs'
gr_file_name='webdocs_00_ground_truth.csv'
grtruth=pd.read_csv(os.path.join(gr_path,gr_file_name))
My_result=pd.read_csv(os.path.join(r'..\result',name))

# precision
gt_set=set(grtruth['Element'][:size])
    # Top-k ground truth
my_set=set(My_result['ID'])
tp_set=gt_set & my_set
    # true-positive set
    # &: set 交集運算
precision=len(tp_set)/len(my_set)
print("Precision: {:8.4f}".format(precision))

# ====================ARE/AAE for all====================
gt_dict=dict(grtruth.values.tolist())
top_dict=dict(My_result.values.tolist())
distinct=len(gt_dict)
    # cardinality of all incoming elements
row_cardinality=[len(i.distinct) for i in Sk_head]
    # number of distinct elements in each row in Sk_head
local_maxID=[i.maxID for i in Sk_head]
    # list of local max elements in each row of Sketch
all_are=0
all_aae=0
top_are=0
top_aae=0
tp=0
fp=0

startx=time.time()
for item in gt_dict:
    count=Query(str(item),Top,local_maxID,Sketch,row_cardinality)
    all_are+=abs(count-gt_dict[item])/gt_dict[item]
    all_aae+=abs(count-gt_dict[item])
    if item in top_dict:
        # compute ARE/AAE of all elements in Top but not only hit elements
        top_are+=abs(count-gt_dict[item])/gt_dict[item]
        top_aae+=abs(count-gt_dict[item])
        if item in tp_set:
            tp+=1
        else:
            fp+=1
endx=time.time()

top_are=top_are/size
top_aae=top_aae/size
all_are=all_are/distinct
all_aae=all_aae/distinct
print("Find:{}, TP:{}, FP:{}".format(len(tp_set),tp,fp))
print("Top_ARE: {:8.6f}".format(top_are))
print("Top_AAE: {:8.6f}".format(top_aae))
print("all_ARE: {:8.6f}".format(all_are))
print("all_AAE: {:8.6f}".format(all_aae))
print("Estimate time:{:8.3f} seconds.".format(endx-startx))        

# ====================Cover result into a dataframe====================
sketch_size=str(depth)+'*'+str(width)
temp=sys.getsizeof(Top)+Sketch.nbytes+sys.getsizeof(Sk_head[0])*depth
memory_usage=str(temp)+' bytes ='+'Top:'+str(sys.getsizeof(Top))+'+ Sketch:'+str(Sketch.nbytes)+'+ Sk_head:'+str(sys.getsizeof(Sk_head[0])*depth)

result_df=pd.DataFrame(columns=['Top-k',
                                'Sketch',
                                'Total memory',
                                'Exe_time',
                                'Find',
                                'Precision',
                                'ARE-Top',
                                'AAE-Top',
                                'ARE-all',
                                'AAE-all'])
output_dict={
    'Top-k':size,
    'Sketch':sketch_size,
    'Total memory':memory_usage,
    'Exe_time':float('{:.3f}'.format(end-start)),
    'Find':"Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp),
    'Precision':float("{:8.4f}".format(precision)),
    'ARE-Top':float('{:8.6f}'.format(top_are)),
    'AAE-Top':float('{:8.6f}'.format(top_aae)),
    'ARE-all':float('{:8.6f}'.format(all_ARE)),
    'AAE-all':float('{:8.6f}'.format(all_AAE))}

result_df=result_df.append(output_dict,ignore_index=True)
file="My_webdocs_result"+'_'+str(size)+'_'+str(depth)+'_'+str(width)+'_.csv'
    # 程式執行結果供複製用
result_df.to_csv(os.path.join(os.path.join(r'..\result',file)),index=False)
result_df

EOF
Top-512,Sketch:128*256
Execution time:1030.936 seconds.
Total memory 142512 bytes=Top:4272 bytes, Sketch:131072 bytes, Sketch_head:7168 bytes.
Precision:   0.9492
Find:486, TP:486, FP:26
Top_ARE: 0.008511
Top_AAE: 57.750000
all_ARE: 10.443191
all_AAE: 25.509736
Estimate time:  35.904 seconds.


,Top-k,Sketch,Total memory,Exe_time,Find,Precision,ARE-Top,AAE-Top,ARE-all,AAE-all
0,512,128*256,142512 bytes =Top:4272+ Sketch:131072+ Sk_head...,1030.936,"Find:486, TP:486, FP:26",0.9492,0.008511,57.75,10.70181,26.593554


In [1]:
Top

NameError: name 'Top' is not defined